# LitMatter DeepChem
* This notebook shows how to speed up [DeepChem](https://github.com/deepchem/deepchem) model training on [MoleculeNet](https://arxiv.org/abs/1703.00564) datasets using the LitMatter template.  
* In this example, we train a simple DeepChem `TorchModel` on the Tox21 dataset.
* The training workflow shown here can be scaled to hundreds of GPUs by changing a single keyword argument!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch

In [3]:
import deepchem as dc

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import (LightningDataModule, LightningModule, Trainer,
                               seed_everything)

### Load a `LitMolNet` dataset
Any MolNet dataset from `deepchem.molnet` can be used with LitMatter. The specific MolNet dataset and any pre-processing steps can be defined in `data.LitMolNet`

In [4]:
from lit_data.molnet_data import LitMolNet

dm = LitMolNet(loader=dc.molnet.load_tox21, batch_size=16)
dm.prepare_data()
dm.setup()

RDKit WARNING: [11:14:30] WARNING: not removing hydrogen atom without neighbors
[11:14:30] WARNING: not removing hydrogen atom without neighbors
RDKit WARNING: [11:14:41] WARNING: not removing hydrogen atom without neighbors
[11:14:41] WARNING: not removing hydrogen atom without neighbors


### Instantiate a `LitDeepChem` model
Any `deepchem.models.torch_models.TorchModel` can be used with LitMatter. Here, we'll write our own custom base model in PyTorch and make a `TorchModel`.

In [9]:
from lit_models.deepchem_models import LitDeepChem

base_model = torch.nn.Sequential(
torch.nn.Linear(1024, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 12),
)

torch_model = dc.models.TorchModel(base_model, loss=torch.nn.MSELoss())

model = LitDeepChem(torch_model, lr=1e-2)

Exception ignored in: <function Model.__del__ at 0x7f028f6c6550>
Traceback (most recent call last):
  File "/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/deepchem/models/models.py", line 61, in __del__
    shutil.rmtree(self.model_dir)
  File "/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/shutil.py", line 709, in rmtree
    onerror(os.lstat, path, sys.exc_info())
  File "/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/shutil.py", line 707, in rmtree
    orig_st = os.lstat(path)
FileNotFoundError: [Errno 2] No such file or directory: '/state/partition1/slurm_tmp/48690281.0.0/tmp60c4fwyb'


### Train the model
Simply change the `Trainer` flags as desired for multi-gpu and multi-node training.

In [11]:
trainer = Trainer(gpus=-1,  # use all available GPUs on each node
#                   num_nodes=1,  # change to number of available nodes
#                  accelerator='ddp',
                 max_epochs=5,
                 )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [12]:
trainer.fit(model, datamodule=dm)

/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [GPU-13636255-d3c9-b0ac-83c7-b25c82e0dbc5]
Set SLURM handle signals.

  | Name    | Type       | Params
---------------------------------------
0 | model   | Sequential | 265 K 
1 | loss_fn | MSELoss    | 0     
---------------------------------------
265 K     Trainable params
0         Non-trainable params
265 K     Total params
1.062     Total estimated model params size (MB)
/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:617: UserWarning: Checkpoint directory /home/gridsan/NA30490/litmatter_dev/litmatter/lightning_logs/version_48690281/checkpoints exists and is not empt

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 0: : 392it [00:01, 225.99it/s, loss=14.3, v_num=4.87e+7]        

/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 8. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(



Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 0: : 401it [00:01, 215.30it/s, loss=14.3, v_num=4.87e+7]
Epoch 0: : 442it [00:01, 223.67it/s, loss=14.3, v_num=4.87e+7, val_loss=19.20]
Epoch 1: : 0it [00:00, ?it/s, loss=14.3, v_num=4.87e+7, val_loss=19.20]       

/home/gridsan/NA30490/.conda/envs/litmatter/lib/python3.8/site-packages/pytorch_lightning/utilities/data.py:56: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 15. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(


Epoch 1: : 392it [00:01, 233.84it/s, loss=9.79, v_num=4.87e+7, val_loss=19.20, train_loss=13.20]
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 1: : 442it [00:01, 235.14it/s, loss=9.79, v_num=4.87e+7, val_loss=19.00, train_loss=13.20]
Epoch 2: : 392it [00:01, 264.65it/s, loss=13.7, v_num=4.87e+7, val_loss=19.00, train_loss=11.00]
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 2: : 442it [00:01, 260.55it/s, loss=13.7, v_num=4.87e+7, val_loss=18.90, train_loss=11.00]
Epoch 3: : 392it [00:01, 261.71it/s, loss=7.03, v_num=4.87e+7, val_loss=18.90, train_loss=8.930]
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 3: : 442it [00:01, 259.93it/s, loss=7.03, v_num=4.87e+7, val_loss=19.30, train_loss=8.930]
Epoch 4: : 392it [00:01, 268.69it/s, loss=6.4, v_num=4.87e+7, val_loss=19.30, train_loss=7.330] 
Validating: 0it [00:00, ?it/s]
Validating: 0it [00:00, ?it/s]
Epoch 4: : 442it [00:01, 267.11it/s, loss=6.4, v_num=4.87e+7, val_loss=19

That's it! By changing the `num_nodes` argument, training can be distributed across all available GPUs. For longer training jobs on an HPC cluster, see the provided example batch scripts.